# emart Icecream Data

#### 1. 필요한 패키지 참조

In [1]:
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time

from bs4 import BeautifulSoup
from pandas import DataFrame

#### 2. Chrome Browser 가동

In [2]:
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()

# 크롬브라우저가 준비될 때 까지 최대 5초씩 대기
driver.implicitly_wait(5)

#### 3. emart 페이지 별 아이스크림에 접근하기 위한 변수값 설정 (총 22 페이지)

In [3]:
params = list(range(1, 23))
url_emart = "https://emart.ssg.com/disp/category.ssg?dispCtgId=6000213403&page={0}"

#### 4. 내용 수집

##### 1) 모든 정보 수집할 리스트 및 판매명에 포함되어 있는 브랜드명이 담긴 리스트 생성

In [4]:
icecream_data = []
brandName = ['빙그레', '해태', 'No Brand', '노브랜드', '롯데', '피코크', 'SSG푸드마켓', '매일유업',
            '나뚜루', '나뚜르', '하겐다즈', '설빙', '바른씨', '아이러브아이스크림', '제이큐',
            '글로벌푸드', '우리밀', '유니레버', '골든벨', '동서', '서울우유', '허쉬', '화과방', 
            '매그넘', '상상앤드', '고디바','우리가스토리', '자연맛남', '앤드류허쉬', '1am',
            '비비수산', '인정식탁', '널담', '몽테이블', '부르스터스', '네추럴킹덤', '오설록', 
            '델몬트', '아이스올리', '라벨리', '라라스윗', '할로', '끌레도르', '벤엔제리스', '상하목장',
            '삼립', '다이센', '네니아', '킷캣', '미닛메이드', '엠즈씨드', '사몬타나', '헤일로탑', 
            '바라톨리노', '마이노멀', '테이트', '하이디', '맥키스','매키스']

##### 2) 페이지 별 아이스크림의 정보가 표시되는 html&css 추출 + 추출된 아이스크림 목록 수 만큼 반복하여 판매명, 브랜드명, 가격 가져오기

In [5]:
# 페이지별 반복
for p in params:
    url = url_emart.format(p)
    driver.get(url)
    time.sleep(1)

    soup = BeautifulSoup(driver.page_source)
    icecreamList = soup.select(".mnemitem_grid_item")
    
    # 추출된 아이스크림 목록 수 만큼 반복
    for i in icecreamList:
        # 판매명
        name = i.select(".mnemitem_goods_tit")
        if name:
            nameValue = name[0].text.strip()
        else: 
            nameValue = None
        
        # 브랜드명
        brand = i.select(".mnemitem_goods_brand")
        if brand:
            brandValue = brand[0].text.strip()
        else:
            for b in brandName:
                if b in nameValue:
                    brandValue = b
                    break
                else:
                    brandValue = None

        # 가격
        price = i.select(".new_price > .ssg_price")
        if price:
            priceValue = price[0].text.strip().replace(',', '')
        else:
            priceValue = None
        
        # 결과 병합
        resultDic = {'판매명': nameValue, 
                     '브랜드': brandValue,
                     '가격(원)': priceValue}
        icecream_data.append(resultDic)

#### 5. Excel 파일로 수집한 내용 저장

In [ ]:
df = DataFrame(icecream_data)
df.to_excel("icecream_list_emart.xlsx")